# <font color="#418FDE" size="6.5" uppercase>**Building Llama 3 Agents**</font>

>Last update: 20260118.
    
By the end of this Lecture, you will be able to:
- Describe how LangChain agents orchestrate Llama 3 reasoning and tool use. 
- Configure a LangChain agent that uses Llama 3 with tools and memory to solve multi-step tasks. 
- Analyze agent traces to debug reasoning errors and refine prompts, tools, or configurations. 


## **1. Agent Planning Basics**

### **1.1. Planner Executor Workflow**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master LangChain & Llama 3/Module_05/Lecture_A/image_01_01.jpg?v=1768770217" width="250">



>* Planner designs step-by-step plan using tools
>* Executor runs the plan, tools, and updates

>* Planner designs structured, multi-step approach to tasks
>* Executor runs tools, Llama 3 refines plan iteratively

>* Planner and executor adapt plans as information changes
>* Dynamic loop handles uncertainty and keeps progress clear



In [ ]:
#@title Python Code - Planner Executor Workflow

# Demonstrate simple planner executor workflow using plain Python structures.
# Show planner creating steps and executor running those steps sequentially.
# Print trace showing planning, execution, and final summarized result.

# pip install commands are unnecessary because this script uses standard libraries only.

# Define a simple planner that creates ordered task steps.
def planner_create_plan(user_goal_description, available_tools_list):
    # Build a high level plan using the described tools.
    plan_steps_list = []
    plan_steps_list.append(f"Understand goal: {user_goal_description}.")
    plan_steps_list.append(f"Collect data using: {available_tools_list}.")
    plan_steps_list.append("Analyze collected data for important patterns.")
    plan_steps_list.append("Summarize findings into clear recommendations.")
    return plan_steps_list

# Define fake tools that the executor can call during execution.
def tool_market_search(topic_description):
    # Pretend to search markets and return short descriptive text.
    return f"Found strong demand for {topic_description} in urban United States regions."

# Define another fake tool for cost estimation using simple arithmetic.
def tool_cost_estimate(units_integer, unit_cost_dollars):
    # Compute total cost using units and unit cost values.
    total_cost_value = units_integer * unit_cost_dollars
    return f"Estimated marketing cost is ${total_cost_value} for {units_integer} units."

# Define the executor that follows the plan and calls tools.
def executor_run_plan(plan_steps_list, user_topic_description):
    # Store intermediate results for later summarization.
    results_dictionary = {}
    print("--- Planning and execution trace start ---")
    print("Plan created by planner:")
    for index_value, step_text in enumerate(plan_steps_list, start=1):
        print(f"Step {index_value}: {step_text}")

    # Execute step two using the market search tool.
    print("Executing step two using market search tool.")
    market_info_text = tool_market_search(user_topic_description)
    results_dictionary["market_info"] = market_info_text

    # Execute step three using the cost estimate tool.
    print("Executing step three using cost estimate tool.")
    cost_info_text = tool_cost_estimate(units_integer=1000, unit_cost_dollars=2)
    results_dictionary["cost_info"] = cost_info_text

    # Simulate Llama reasoning that summarizes the collected information.
    print("Executing final step summarizing collected information.")
    summary_text = (
        f"For product '{user_topic_description}', {market_info_text} "
        f"Additionally, {cost_info_text}"
    )
    results_dictionary["summary"] = summary_text

    print("--- Final agent answer ---")
    print(summary_text)
    return results_dictionary

# Main block that connects planner and executor for demonstration.
if __name__ == "__main__":
    # Define user goal and available tools for the planner.
    user_goal_description = "launching a new fitness tracker in major cities"
    available_tools_list = ["market_search", "cost_estimate"]

    # Planner creates a plan using the goal and tools list.
    plan_steps_list = planner_create_plan(user_goal_description, available_tools_list)

    # Executor runs the plan and prints the trace and final answer.
    executor_results_dictionary = executor_run_plan(plan_steps_list, "fitness tracker")



### **1.2. Tracing Thoughts and Actions**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master LangChain & Llama 3/Module_05/Lecture_A/image_01_02.jpg?v=1768770250" width="250">



>* Tracing reveals the agent’s hidden reasoning steps
>* Shows tools used, inputs, outputs, and decisions

>* Trace alternates between reasoning text and tool calls
>* Tool results feed next steps until final answer

>* Tracing reveals reasoning gaps and tool misuse
>* Insights from traces guide refinements for reliability



In [ ]:
#@title Python Code - Tracing Thoughts and Actions

# Demonstrate simple agent thoughts and actions tracing conceptually.
# Show alternating thinking steps and tool actions clearly.
# Help beginners visualize invisible reasoning sequences.
# pip install langchain-openai langchain-community.

# Define a simple tool that converts miles to kilometers.
def convert_miles_to_km(miles_value):
    km_value = miles_value * 1.60934
    return km_value

# Define a function that simulates agent reasoning steps.
def plan_road_trip(distance_miles, fuel_mpg):
    thoughts_and_actions = []
    thoughts_and_actions.append("Thought: I need total fuel gallons.")

    gallons_needed = distance_miles / fuel_mpg
    thoughts_and_actions.append(f"Action: Calculated gallons needed: {gallons_needed:.2f}.")

    thoughts_and_actions.append("Thought: I want distance kilometers also.")
    km_distance = convert_miles_to_km(distance_miles)
    thoughts_and_actions.append(f"Action: Tool converted miles to kilometers: {km_distance:.1f}.")

    summary = f"Final: Drive {distance_miles} miles using {gallons_needed:.1f} gallons."
    thoughts_and_actions.append(summary)
    return thoughts_and_actions

# Run the simulated agent and print traced steps.
trace_steps = plan_road_trip(distance_miles=300, fuel_mpg=25)
for step in trace_steps:
    print(step)



### **1.3. When to use agents**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master LangChain & Llama 3/Module_05/Lecture_A/image_01_03.jpg?v=1768770273" width="250">



>* Use agents when models must choose tools autonomously
>* Skip agents for simple, single-step Llama tasks

>* Use agents for branching, exploratory, data-gathering tasks
>* Agent iteratively chooses tools and stops when confident

>* Use simple workflows for fixed, predictable tasks
>* Use agents when autonomy and adaptability add value



## **2. Configuring Llama Agents**

### **2.1. Choosing Agent Tools**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master LangChain & Llama 3/Module_05/Lecture_A/image_02_01.jpg?v=1768770296" width="250">



>* Chosen tools define what the agent can do
>* Start from tasks and pick minimal, purposeful tools

>* Choose between coarse and fine-grained tools
>* Balance simplicity, flexibility, cost, and reliability

>* Prioritize safe, reliable, clearly described agent tools
>* Use structure, sandboxes, and humans for risky actions



### **2.2. Adding conversational memory**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master LangChain & Llama 3/Module_05/Lecture_A/image_02_02.jpg?v=1768770318" width="250">



>* Conversational memory lets agents keep ongoing context
>* Decide what to store, retain, and summarize

>* Memory component retrieves, compresses, and injects context
>* Keeps user constraints for coherent, personalized multi-step tasks

>* Store only relevant, necessary conversation details
>* Segment, summarize, and control memory for safety



In [ ]:
#@title Python Code - Adding conversational memory

# Demonstrate simple conversational memory with a tiny mock Llama agent.
# Show difference between no memory and basic summary style memory.
# Keep everything beginner friendly and runnable inside Google Colab.
# pip install langchain llama-cpp-python sentence-transformers transformers.

# Import required standard library modules for simple data handling.
from typing import List, Dict, Any

# Define a simple class representing conversational memory storage.
class SimpleMemory:

    # Initialize memory with empty conversation history and user facts.
    def __init__(self) -> None:
        self.turn_history: List[Dict[str, str]] = []
        self.user_facts: Dict[str, str] = {}

    # Store a new conversation turn with role and content text.
    def add_turn(self, role: str, content: str) -> None:
        self.turn_history.append({"role": role, "content": content})

    # Update long term user facts based on simple keyword rules.
    def update_facts(self, user_message: str) -> None:
        lower = user_message.lower()
        if "morning" in lower:
            self.user_facts["preferred_time"] = "morning classes preferred"
        if "part-time" in lower:
            self.user_facts["work_status"] = "works part-time during semester"
        if "statistics" in lower:
            self.user_facts["weak_subject"] = "needs extra help with statistics"

    # Build a short memory summary string for the model context.
    def build_summary(self) -> str:
        facts_parts = [f"- {k}: {v}" for k, v in self.user_facts.items()]
        if not facts_parts:
            return "No stable user preferences remembered yet."
        joined = "\n".join(facts_parts)
        return f"Known user preferences and constraints so far:\n{joined}"


# Define a tiny mock Llama model that uses context strings.
class MockLlamaModel:

    # Initialize model with a simple name for identification.
    def __init__(self, name: str = "MockLlama") -> None:
        self.name = name

    # Generate a reply using current message and memory summary.
    def generate(self, message: str, memory_summary: str) -> str:
        if "schedule" in message.lower():
            return "I will design a schedule using remembered preferences and constraints."
        if "remind" in message.lower():
            return f"Here are preferences I remember now:\n{memory_summary}"
        return "I noted your message and updated my remembered preferences."


# Define an agent that connects user messages, memory, and mock model.
class LlamaAgentWithMemory:

    # Initialize agent with memory component and mock model instance.
    def __init__(self) -> None:
        self.memory = SimpleMemory()
        self.model = MockLlamaModel()

    # Handle a user message, updating memory and generating response.
    def handle_message(self, user_message: str) -> str:
        self.memory.add_turn("user", user_message)
        self.memory.update_facts(user_message)
        summary = self.memory.build_summary()
        reply = self.model.generate(user_message, summary)
        self.memory.add_turn("assistant", reply)
        return reply


# Demonstrate conversation without memory by ignoring previous messages.
def conversation_without_memory(messages: List[str]) -> None:
    model = MockLlamaModel()
    for msg in messages:
        reply = model.generate(msg, memory_summary="No previous context used here.")
        print(f"User: {msg}")
        print(f"Agent: {reply}")


# Demonstrate conversation with memory using the LlamaAgentWithMemory.
def conversation_with_memory(messages: List[str]) -> None:
    agent = LlamaAgentWithMemory()
    for msg in messages:
        reply = agent.handle_message(msg)
        print(f"User: {msg}")
        print(f"Agent: {reply}")


# Prepare a short scenario about planning a semester schedule.
messages_sequence = [
    "I work part-time and prefer morning classes if possible.",
    "I usually struggle with statistics and heavy math courses.",
    "Can you suggest a weekly schedule for my semester now?",
    "Please remind me what preferences you are using today.",
]

# Run both conversations and clearly separate their printed outputs.
print("--- Conversation without conversational memory enabled ---")
conversation_without_memory(messages_sequence)
print("--- Conversation with conversational memory enabled ---")
conversation_with_memory(messages_sequence)



### **2.3. Effective Agent Prompting**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master LangChain & Llama 3/Module_05/Lecture_A/image_02_03.jpg?v=1768770353" width="250">



>* Define clear role, goal, and behavior instructions
>* Require tool use for facts, forbid making up data

>* Tell the agent when and how to use tools
>* Guide memory use to connect steps into plans

>* Prompt agents to show structured, checkable reasoning steps
>* Use reflective prompts to improve traces and robustness



In [ ]:
#@title Python Code - Effective Agent Prompting

# Demonstrate effective agent prompting using simple rule based reasoning simulation.
# Show how role, goals, and steps change reasoning behavior clearly.
# Compare vague prompt versus structured prompt for a planning style assistant.

# pip install langchain llama3 hypothetical packages if actually using them.

# Define a function simulating an agent following vague instructions.
def vague_agent_response(user_request, tools_available, memory_notes):
    # Build a loose style response ignoring tools and memory mostly.
    response = "I will try something quickly without checking details carefully. "
    response += f"Request: {user_request}. "
    response += "I guess things based on general knowledge and imagination. "
    response += "I might forget earlier constraints or user preferences."
    return response


# Define a function simulating an agent following structured instructions.
def structured_agent_response(user_request, tools_available, memory_notes):
    # Start with explicit role and goal description.
    role = "You are a careful academic planning assistant."
    goal = "Your goal is building a feasible weekly study plan."
    # Describe explicit tool usage expectations clearly.
    tool_instruction = "Use tools whenever information seems missing or uncertain."
    # Describe memory usage expectations clearly and explicitly.
    memory_instruction = "Always restate important preferences from memory before deciding."
    # Simulate tool usage by referencing provided tool names.
    used_tools = f"Tools consulted: {', '.join(tools_available)}."
    # Simulate memory recall by summarizing memory notes briefly.
    memory_summary = f"Key remembered preferences: {memory_notes}."
    # Simulate stepwise reasoning with clear ordered steps.
    steps = [
        "Step 1: Restate user goal and constraints clearly.",
        "Step 2: Call schedule_lookup tool for available time slots.",
        "Step 3: Call prerequisite_checker tool for course requirements.",
        "Step 4: Combine tool results with remembered preferences.",
        "Step 5: Propose schedule and verify no conflicts remain."
    ]
    # Join everything into one coherent explanation string.
    response_parts = [role, goal, tool_instruction, memory_instruction, used_tools, memory_summary]
    response_parts.extend(steps)
    response = " \n".join(response_parts)
    return response


# Define a simple scenario representing a multi step planning request.
user_request = "Plan a weekly study schedule for physics and history courses."
# Define available tools names representing external information sources.
tools_available = ["schedule_lookup", "prerequisite_checker"]
# Define conversational memory notes representing user preferences.
memory_notes = "Prefers evenings, avoids weekends, needs two hours daily minimum."

# Generate responses from both vague and structured simulated agents.
vague_output = vague_agent_response(user_request, tools_available, memory_notes)
structured_output = structured_agent_response(user_request, tools_available, memory_notes)

# Print both outputs to compare prompting effects clearly and concisely.
print("VAGUE PROMPT STYLE OUTPUT:\n")
print(vague_output)
print("\nSTRUCTURED PROMPT STYLE OUTPUT:\n")
print(structured_output)



## **3. Debugging Llama Agents**

### **3.1. Tracing Agent Reasoning**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master LangChain & Llama 3/Module_05/Lecture_A/image_03_01.jpg?v=1768770394" width="250">



>* Tracing reveals each step of agent reasoning
>* Chronological events explain what the agent did

>* Check intent, tool choice, and parameters carefully
>* Compare tool outputs to responses to spot divergences

>* Link recurring trace patterns to design changes
>* Use traces as diagnostics to refine agents



In [ ]:
#@title Python Code - Tracing Agent Reasoning

# Demonstrate simple agent reasoning trace with printed steps.
# Show how an agent chooses tools and explains decisions.
# Help beginners connect traces with debugging agent behavior.

# pip install langchain-openai langchain-community.
# pip install langchain-core langchain-text-splitters.

# Import required standard modules for this simple trace simulation.
import random

# Define a simple user request representing a customer support question.
user_request = "Where is my package and when will it arrive?"

# Define two simple tools with names and short descriptions.
tools = [
    {"name": "order_lookup", "description": "Find order status using order id."},
    {"name": "faq_search", "description": "Search common shipping questions database."},
]

# Define a function that simulates an agent reasoning trace.
def run_agent_with_trace(request_text):
    trace_events = []
    trace_events.append({"event": "user_message", "content": request_text})
    intent = "track_package_and_eta"
    trace_events.append({"event": "model_plan", "content": f"Identify intent as {intent}."})

    chosen_tool = tools[0]
    tool_input = {"order_id": "12345"}
    trace_events.append({"event": "tool_call", "content": f"Call {chosen_tool['name']} with {tool_input}."})

    tool_output = {"status": "In transit", "eta_days": 3}
    trace_events.append({"event": "tool_result", "content": str(tool_output)})

    final_answer = "Your package is in transit and should arrive within three days." 
    trace_events.append({"event": "final_answer", "content": final_answer})
    return trace_events

# Run the simulated agent and capture the reasoning trace.
trace = run_agent_with_trace(user_request)

# Print the trace in chronological order with clear labels.
print("AGENT REASONING TRACE:\n")
for step_number, event in enumerate(trace, start=1):
    print(f"Step {step_number}: {event['event']} -> {event['content']}")



### **3.2. Common Agent Failures**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master LangChain & Llama 3/Module_05/Lecture_A/image_03_02.jpg?v=1768770434" width="250">



>* Agents often misuse tools or loop without progress
>* Fix by clarifying tool descriptions and usage rules

>* Reasoning can seem correct but hide subtle errors
>* These errors mislead actions and require instruction tweaks

>* Agents often mishandle memory and long-term context
>* Spot anomalies in traces to tune memory



### **3.3. Iterative refinement cycles**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master LangChain & Llama 3/Module_05/Lecture_A/image_03_03.jpg?v=1768770453" width="250">



>* Start from a specific failure in traces
>* Change one configuration element per refinement cycle

>* Re-test the agent on similar scenarios
>* Compare traces, watch tradeoffs, plan next tweaks

>* Iterative feedback cycles create reliable, predictable agents
>* Use trace patterns to apply targeted refinements



# <font color="#418FDE" size="6.5" uppercase>**Building Llama 3 Agents**</font>


In this lecture, you learned to:
- Describe how LangChain agents orchestrate Llama 3 reasoning and tool use. 
- Configure a LangChain agent that uses Llama 3 with tools and memory to solve multi-step tasks. 
- Analyze agent traces to debug reasoning errors and refine prompts, tools, or configurations. 

<font color='yellow'>Congratulations on completing this course!</font>